In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from numpy import mean
from numpy import std
from mlxtend.evaluate import paired_ttest_5x2cv,combined_ftest_5x2cv
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
from sklearn.neighbors import LocalOutlierFactor

# evaluate label propagation on the semi-supervised learning dataset

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']
Covariance=['spherical', 'tied', 'diag', 'full']
Params=['kmeans', 'random']
skf = StratifiedKFold(n_splits=2) 
fold=0
test=False
for train_index, test_index in skf.split(X, y):
        fold=fold+1
        k = 0
        liste =[0]*8
        
        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        for i in Covariance:
                for j in Params:
                       
                       
                        gmm = GaussianMixture(n_components=2,init_params=j,covariance_type=i)
                        gmm.fit(X_train,y_train)
                        y_pred=gmm.predict(X_test)
                        if precision_score(y_test,y_pred)< 0.1:
    
                                y_pred = np.array([0 if label == 1 else 1 for label in y_pred])
                        
                        F1=round(f1_score(y_test,y_pred),3)
                        recall=round(recall_score(y_test,y_pred),3)
                        precision=round(precision_score(y_test,y_pred,zero_division=0),3)
                        accuracy=round(accuracy_score(y_test,y_pred),3)
                        fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred)
                        roc_auc = metrics.auc(fpr, tpr)
                        if test==False :
                                data = np.array([[i,j,F1,recall,precision,accuracy,roc_auc,fold]])
                                test=True
                        else :
                                data = np.append(data, [[i,j,F1,recall,precision,accuracy,roc_auc,fold]],axis=0)
                        liste[k]=gmm
                        k = k+1
                       
        for estimator1, estimator2 in zip(liste,liste[1:]):
                t, p = paired_ttest_5x2cv(estimator1= estimator1, estimator2=estimator2,X=X, y=y, scoring='f1', random_seed=None)
        
                print('P-value: %.3f, t-Statistic: %.3f' % (p, t))

              
                        
             


P-value: 0.130, t-Statistic: 1.812
P-value: 0.314, t-Statistic: -1.119
P-value: 0.341, t-Statistic: 1.053
P-value: 0.468, t-Statistic: -0.785
P-value: 1.000, t-Statistic: 0.000
P-value: 0.989, t-Statistic: 0.015
P-value: 0.127, t-Statistic: 1.830
P-value: 0.076, t-Statistic: -2.236
P-value: 0.169, t-Statistic: -1.606
P-value: 0.349, t-Statistic: 1.034
P-value: 0.729, t-Statistic: -0.366
P-value: 0.251, t-Statistic: 1.298
P-value: 0.993, t-Statistic: 0.009
P-value: 1.000, t-Statistic: 0.000


In [17]:
import pandas as pd
from scipy.stats import mannwhitneyu
from statsmodels.formula.api import ols

df = pd.DataFrame(data=data,columns=['Covariance','Parametre','F1','recall','precision','accuracy','AUC','numero_fold'])
df


,Covariance,Parametre,F1,recall,precision,accuracy,AUC,numero_fold
0,spherical,kmeans,0.905,0.992,0.832,0.927,0.9417792792792793,1
1,spherical,random,0.905,0.992,0.832,0.927,0.9417792792792793,1
2,tied,kmeans,0.905,0.875,0.938,0.936,0.9217342342342342,1
3,tied,random,0.307,0.417,0.243,0.339,0.35698198198198194,1
4,diag,kmeans,0.862,0.992,0.763,0.889,0.9125000000000001,1
5,diag,random,0.862,0.992,0.763,0.889,0.9125000000000001,1
6,full,kmeans,0.811,1.0,0.682,0.836,0.8738738738738738,1
7,full,random,0.811,1.0,0.682,0.836,0.8738738738738738,1
8,spherical,kmeans,0.948,1.0,0.902,0.962,0.9707207207207208,2
9,spherical,random,0.948,1.0,0.902,0.962,0.9707207207207208,2


In [18]:
estimateur1 = GaussianMixture(n_components=2, covariance_type = 'full', init_params='kmeans')
estimateur2 = LocalOutlierFactor(n_neighbors=round(len(X)*0.625),contamination=0.357,metric='euclidean')
t,p = paired_ttest_5x2cv(estimator1=estimateur1,estimator2=estimateur2,X=X, y=y, scoring='f1', random_seed=None)
print('P-value: %.3f, t-Statistic: %.3f' % (p, t))

c:\Users\junda\anaconda3\lib\site-packages\sklearn\neighbors\_lof.py:274: UserWarning: n_neighbors (427) is greater than the total number of samples (341). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn("n_neighbors (%s) is greater than the "


AttributeError: predict is not available when novelty=False, use fit_predict if you want to predict on training data. Use novelty=True if you want to use LOF for novelty detection and predict on new unseen data.